# TFY4345 - Classical Mechanics - Numerical Exercise

### Alexander Arntzen | Knut Andre G. Prestsveen

This is a project in the course TFY4345 "Classical Mechanics" at the Norwegian University of Science and Technology. The notebook studies a simple pendulum, comparing the results of different numerical methods for solving the pendulum's equation of motion.

The source code lies on [GitHub](https://github.com/kaprests/Pendulum).

In [ ]:
'''Imports packages, functions and constants'''
# Simple pendulum with Eulers method:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib.animation import FuncAnimation
from scipy.constants import g
from scipy.integrate import solve_ivp
from IPython.display import HTML

In [ ]:
# Set common figure parameters:
newparams = {'axes.labelsize': 11, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (8, 3),
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,}
plt.rcParams.update(newparams)

In [ ]:
'''Total energy of a pendulum'''
def energies(theta, omg):
    '''
    Calculates the kinetic, potential and total energies of a pendulum.
    
    Input:
        theta: angle between the pendulum and the vertical axis.
        omg: the angular velocity of the pendulum.
    Returns:
        T: the pendulum's kinetic energy.
        V: the pendulum's potential energy.
        E_tot: the pendulum's total energy.
    '''
    T = (1/2)*m*(l**2)*omg**2
    V = m*g*l*(1 - np.cos(theta))
    E_tot = T + V
    return T, V, E_tot

# Assignment 2 

## Task 1 


In [ ]:
'''Oppgave 1'''
#Initial conditions
m = 1 #kg 
l = 1 # m 
theta_0 = 0.2 # rad 
omega_0 = 0.0 # rad 
q = 1.0 # s^-1
F_d = 0.2 # s^-2
sim_time = 20 #s, simulation time
dt = 0.1 # s, timestep
omg_d = 3.3 # driving freq. for forced oscillator


In [ ]:
def df(t, y,q=q,g=g,l=l): 
    '''
    Defines the equation of motion as a pair of
    first order equaitions.
    '''
    return [y[1],-g/l*y[0]-q*y[1] ]


def theta_analytic(t,theta_0=theta_0,q=q,g=g,l=l):
    '''
    Analytical solution to the dampened oscillator
    
    input:
        t: time
        theta_0: initial position (angle)
        q: dampening coefficient
        g: gravitational acceleration
        l: length of pendulum
    output:
        The analytical solution
    '''
    if q < np.sqrt(4*g/l):
        return theta_0*np.exp(-q*t*0.5)*np.cos(np.sqrt((g/l)- q**2/4)*t)+ theta_0*q/(2*np.sqrt((g/l)- q**2/4))*np.exp(-q*t*0.5)*np.sin(np.sqrt((g/l)- q**2/4)*t)
    elif q == np.sqrt(4*g/l):
        return (theta_0+theta_0*q*0.5*t)*np.exp(-q*t/2)
    else:
        return (-q/2 + np.sqrt(q**2/4 - g/l))/(2*np.sqrt(q**2/4 - g/l))*theta_0*np.exp((-q/2 - np.sqrt(q**2/4 - g/l))*t) + (q/2 + np.sqrt(q**2/4 - g/l))/(2*np.sqrt(q**2/4 - g/l))*theta_0*np.exp((-q/2 + np.sqrt(q**2/4 - g/l))*t)




In [ ]:
'''Underdamped'''
q_under = np.sqrt(1*g/l)

sol_under = solve_ivp(lambda t, y: df(t,y,q_under), t_span=[0, sim_time], y0=[theta_0, omega_0], max_step = dt )
theta_vec_under, omg_vec_under = sol_under.y[0], sol_under.y[1]
kin_energy_under, pot_energy_under, total_energy_under = energies(sol_under.y[0], sol_under.y[1])

"""plot results"""
plt.title("Underdamped")
plt.plot(sol_under.t, theta_vec_under, label = "Angle numeric ")
plt.plot(sol_under.t, theta_analytic(sol_under.t, q = q_under ), label = "Angle analytic")
plt.xlabel("Time [s]")
plt.ylabel("Angle [rad]")
plt.legend()
plt.show()

plt.plot(sol_under.t, total_energy_under[0] - total_energy_under , label = "Work done by dampening ")
#plt.plot(sol.t, total_energy_RK45, label = "Work done by dampening analytical")
plt.xlabel("Time [s]")
plt.ylabel("Energy [J]")
plt.legend()
plt.show()

plt.show()


The analytical and numerical solutions are plottet on top of each other for a underdamped system. The solutions mach well.

In [ ]:
'''Critically damped'''
q_crit = np.sqrt(4*g/l)
sol_crit = solve_ivp(lambda t, y: df(t,y,q_crit), t_span=[0, sim_time], y0=[theta_0, omega_0], max_step = dt )
theta_vec_crit, omg_vec_crit = sol_crit.y[0], sol_crit.y[1]
kin_energy_crit, pot_energy_crit, total_energy_crit = energies(sol_crit.y[0], sol_crit.y[1])

#plot results
plt.title("critdamped")
plt.plot(sol_crit.t, theta_vec_crit, label = "Angle numeric ")

plt.plot(sol_crit.t, theta_analytic(sol_crit.t, q = q_crit ), label = "Angle analytic")
plt.xlabel("Time [s]")
plt.ylabel("Angle [rad]")
plt.legend()
plt.show()

plt.plot(sol_crit.t, total_energy_crit[0] - total_energy_crit , label = "Work done by dampening ")
#plt.plot(sol.t, total_energy_RK45, label = "Work done by dampening analytical")
plt.xlabel("Time [s]")
plt.ylabel("Energy [J]")
plt.legend()
plt.show()

The analytical and numerical solutions are plottet on top of each other for a critically damped system. The solutions mach well and we can see that the sysem converges to zero the fastest. 

In [ ]:
#Overdamped 
q_over = np.sqrt(8*g/l)
sol_over = solve_ivp(lambda t, y: df(t,y,q_over), t_span=[0, sim_time], y0=[theta_0, omega_0], max_step = dt )
theta_vec_over, omg_vec_over = sol_over.y[0], sol_over.y[1]
kin_energy_over, pot_energy_over, total_energy_over = energies(sol_over.y[0], sol_over.y[1])

"""plot results"""
plt.title("Overdamped")
plt.plot(sol_over.t, theta_vec_over, label = "Angle numeric ")
plt.plot(sol_over.t, theta_analytic(sol_over.t, q = q_over ), label = "Angle analytic")
plt.xlabel("Time [s]")
plt.ylabel("Angle [rad]")
plt.legend()
plt.show()

plt.plot(sol_over.t, total_energy_over[0] - total_energy_over , label = "Work done by dampening ")
#plt.plot(sol.t, total_energy_RK45, label = "Work done by dampening analytical")
plt.xlabel("Time [s]")
plt.ylabel("Energy [J]")
plt.legend()
plt.show()

The analytical and numerical solutions are plottet on top of each other for an overdamped system. The solutions mach well. 

## Task 2 Forced harmonic pendulum

In [ ]:
def ddt_omg_theta(t, y, omg_d=omg_d, q=q, F_d=F_d, g=g, l=l): 
    '''
    Functions defining the eqn. of motion as a pair of
    first order eqns.
    '''
    return [y[1], -g/l*y[0] - q*y[1] + F_d*np.sin(omg_d*t)]


def steady_amp_analytical(omg_d=omg_d, F_d=F_d, q=q):
    return F_d/(np.sqrt((np.sqrt(g/l)**2 - omg_d**2)**2 + (q*omg_d)**2))


def particular(t, theta_0=theta_0, omg_d=theta_0, q=q, g=g, l=1,F_d=F_d):
    return np.sin(omg_d*t-np.pi/2+np.arctan((g/l-omg_d**2)/(omg_d*q)) )*F_d/np.sqrt((g/l-omg_d**2)**2+ (q*omg_d)**2)


In [ ]:
'''Apply the solution'''
sol = solve_ivp(lambda t, y: ddt_omg_theta(t,y, omg_d=omg_d, q=q), t_span=[0, sim_time], y0=[theta_0, omega_0], max_step = dt )

In [ ]:
'''Plot the results'''
plt.plot(sol.t, sol.y[0], ".",label = "Angle numeric ")
plt.plot(sol.t, +theta_analytic(sol.t) +particular(sol.t , omg_d=omg_d), label="Analytic solution")
plt.xlabel("Time [s]")
plt.ylabel("Angle [rad]")
plt.legend()
plt.show()

In the graph above the evolution of the angel $\theta$ is plotted over time. We see that it has a periodic steady state with the same frequency as the driving frequency $\Omega_D$. 

In [ ]:
n = 6 # Num qs
m = 30 # Num omgDs
omg_d_ref = np.sqrt(g/l) # analytical val
qs = np.geomspace(1, 5, n)
omg_ds = np.linspace(0.5*omg_d_ref,3*omg_d_ref, m)
omg_ds_rel = omg_ds/omg_d_ref


steady_amps = np.zeros([n,m])#[0]*n
#steady_amps_q = [0]*n

for i in range(len(qs)): 
    for j in range(len(omg_ds)):
        sol = solve_ivp(lambda t, y: ddt_omg_theta(t, y , omg_d=omg_ds[j], q=qs[i]), t_span=[0, sim_time], y0=[theta_0, omega_0], max_step = dt )
        steady_amps[i][j] = np.max(sol.y[0][3*len(sol.y[0])//4:])

for i in range(len(qs)):
    plt.plot(omg_ds_rel, steady_amps[i], label="q = " + str(qs[i]))
plt.title("Steady state amplitude of driving frequencies ")
plt.xlabel("omega_D/omega [1]")
plt.ylabel("Resonant amplitude[rad]")
plt.legend()
plt.show()

for i in range(len(qs)):
    plt.plot(omg_ds_rel, steady_amp_analytical(omg_d=omg_ds, q=qs[i]), label="q = " + str(qs[i]))
plt.title("Steady state amplitude of driving frequencies ")
plt.xlabel("omega_D/omega [1]")
plt.ylabel("Resonant amplitude[rad]")
plt.legend()
plt.show()



The resonant frequency is plottet above for different values of $q$ and $\Omega_D$. Here we se that the resonant amplitude indeed depends on $q$ and $\Omega_D$. Resonance happens  the the resonant amplitude has a local maximum. This is observed for $q = 0$ and $\Omega_D = \omega$, where $\omega$ is the natural frequency. Thus we have shown existance of resonance.